<a href="https://colab.research.google.com/github/hunterkasparian/DiffusionGraph/blob/main/Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
from scipy.linalg import solve_banded
import plotly.graph_objects as go
from typing import no_type_check
import random

In [301]:
nx = 100 # length
nt = 6000 # time steps
T = np.full(nx,0)
T[0] = 1000 # end concentrations are fixed
x = np.arange(nx) # x axis

fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=x, y = T, mode= 'lines', name='Initial')


for i in range(1,nt):
    pright = np.zeros(nx-2) # probability array for right movement
    pleft = np.zeros(nx-2) # probability array for left movement

    for n in range(1,len(T)-1): # calculating probability of movement ~random~ evaluated per position
      if T[n+1] != 0: # cant divide by zero :/
        pright[n-1] = np.sum ( np.random.choice ([0,1] , (T[n+1]) ) ) / T[n+1] # movement prob = sum (# particles that move (value = 1)) / total particles 
      if T[n-1] != 0:
        pleft[n-1] = np.sum ( np.random.choice ([0,1] , (T[n-1]) ) ) / T[n-1]

    T[1:-1] = pright * T[2:] + pleft * T[:-2] # will never update ends

    if i % 600 == 0:
      fig.add_scatter(x=x, y = T, mode= 'lines', name=f'{i}')
fig.show()
print(T)

[1000  992  928 1010  953  962  936  936  908  925  891  905  866  854
  858  844  841  777  821  774  841  745  794  648  781  627  802  551
  896  559  827  581  782  606  770  524  804  504  798  503  692  502
  668  500  681  450  605  430  600  368  568  353  538  325  486  297
  430  313  412  300  409  294  387  240  331  242  304  244  297  266
  247  245  277  215  263  177  245  162  231  157  208  120  158  121
  130  103  108   95  106   80   71   63   40   45   24   29   20   13
    6    0]
